#  DDP

### 1. running on our DGX
- docker run -it --rm --gpus '"device=4,5,6,7"' -p 12344:12344 --name msalehjahromi --shm-size=192G  --user $(id -u):$(id -g) --group-add 1944259512 --cpuset-cpus=49-96 -v /rsrch7/home/ip_rsrch/wulab/:/rsrch7/home/ip_rsrch/wulab -v /rsrch1/ip/msalehjahromi/:/rsrch1/ip/msalehjahromi --name mori_jupyter nnunetv2:msalehjahromi
- cd /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100
- jupyter notebook --ip 0.0.0.0 --port 12344
- http://1mcprddgx05

In [2]:
!gpustat

jupyter-msalehjahromi     Tue Jun 10 13:41:56 2025  550.54.15
[0] NVIDIA A100-SXM4-40GB | 29'C,   0 % |   637 / 40960 MB |
[1] NVIDIA A100-SXM4-40GB | 31'C,   0 % |   637 / 40960 MB |
[2] NVIDIA A100-SXM4-40GB | 31'C,   0 % |   637 / 40960 MB |
[3] NVIDIA A100-SXM4-40GB | 45'C,   0 % |   637 / 40960 MB |


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

import torch
print(torch.cuda.device_count())   # → 2
print(torch.cuda.current_device()) # → 0 (this maps to your original GPU 2)


4
0


### 2. Probably need installation, restart after and run 3

In [3]:
## !pip uninstall -q histolab -y
# Install required packages first
import os
import subprocess

pip_commands = [
    ["pip", "install","-q", "--extra-index-url", "https://download.pytorch.org/whl/cu117", 
     "torch==2.0.0", "torchvision==0.15.0", "omegaconf", "torchmetrics==0.10.3", 
     "fvcore", "iopath", "xformers==0.0.18", "submitit", "numpy<2.0"],
    ["pip", "install", "-q",  "--extra-index-url", "https://pypi.nvidia.com", "cuml-cu11"],
    ["pip", "install","-q",  "black==22.6.0", "flake8==5.0.4", "pylint==2.15.0"],
    ["pip", "install", "-q", "mmsegmentation==0.27.0"],
    ["pip", "install","-q", "mmcv-full==1.5.0"],
    ["pip", "install","-q", "nibabel"]
]

for cmd in pip_commands:
    try:
        print(cmd)
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to install packages with command: {cmd}")
        print(f"Error: {e}")

['pip', 'install', '-q', '--extra-index-url', 'https://download.pytorch.org/whl/cu117', 'torch==2.0.0', 'torchvision==0.15.0', 'omegaconf', 'torchmetrics==0.10.3', 'fvcore', 'iopath', 'xformers==0.0.18', 'submitit', 'numpy<2.0']
['pip', 'install', '-q', '--extra-index-url', 'https://pypi.nvidia.com', 'cuml-cu11']
['pip', 'install', '-q', 'black==22.6.0', 'flake8==5.0.4', 'pylint==2.15.0']
['pip', 'install', '-q', 'mmsegmentation==0.27.0']
['pip', 'install', '-q', 'mmcv-full==1.5.0']
['pip', 'install', '-q', 'nibabel']


### 3. running, notice important hyperparams
-  which model are use so vmin -1000., vmax +150, eps 0.00005
- `num-workers`
- `epochs`: 100
- `accum-steps`: 2000 ?
- `max-chunks`: 66 
- `lr`:  lr*0.1 on base ?
- `warmup-steps`: 5k
- `print-every` : 5000
- `val-every` : 40000

In [ ]:
n_GPU = "4"
# Set required environment variables
import os
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = n_GPU # "3"
os.environ["LOCAL_RANK"] = "0"
os.environ["LOCAL_WORLD_SIZE"] = n_GPU #  "2"
# Build the command with --install-packages flag removed
command = [
    "python3",
    "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/2_run_fineTune_ddp_launcher.py", ##changed
    "--num-gpus", n_GPU, # ,
    "--csv", "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv",
    "--accum-steps", "500", ###############
    "--num-workers", "3",
    "--epochs", "100", ###############
    "--lr", "0.0001", ###############
    "--warmup-steps", "10", ##########
    "--weight-decay", "0.01",
    "--optimizer", "adamw",
    "--num-attn-heads", "3",
    "--num-layers", "2",
    "--dropout", "0.35",
    "--unfreeze-strategy", "all",  
    "--output", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/output_ddp_p16_23_1150_ac200_JH_d10_B64",
    "--print-every", "1000", ################ 2000
    "--val-every", "41000", ################ 50k
    "--metrics-dir", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/metrics_multi_gpu"
        ]
# Run the command
try:
    import subprocess
    subprocess.run(command, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")
    print(f"Command output: {e.output if hasattr(e, 'output') else 'No output available'}")

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<00:38, 3.97kB/s]
US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<00:41, 3.67kB/s]
US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<00:44, 3.48kB/s]
US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<00:43, 3.57kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0%|          | 125/511k [00:00<01:18, 6.54kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0%|          | 125/511k [00:00<01:33, 5.46kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0%|          | 125/511k [00:00<01:30, 5.62kB/s]
/usr/local/lib/python3.10/dist-packages/pydicom/data/data_manager.py:375: UserWarning: A download failure occurred while attempting to retrieve MR-SIEMENS-DICOM-WithOverlays.dcm
  warn_and_lo

Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']
Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']
Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']
Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.D

2025-06-10 13:43:20,033 - INFO - Starting training on 4 GPUs with full model copies (DDP)


Train dataset created on rank 
Train dataset created on rank 
Train dataset created on rank Train dataset created on rank 


==== Dataset Statistics ====
Total samples: 43852
Training samples: 38382
Validation samples: 5470

1-year-cancer:
  Train positive: 1428 (3.72%)
  Val positive: 188 (3.44%)

2-year-cancer:
  Train positive: 1891 (4.93%)
  Val positive: 256 (4.68%)

3-year-cancer:
  Train positive: 2282 (5.95%)
  Val positive: 315 (5.76%)

4-year-cancer:
  Train positive: 2586 (6.74%)
  Val positive: 363 (6.64%)

5-year-cancer:
  Train positive: 2838 (7.39%)
  Val positive: 403 (7.37%)

6-year-cancer:
  Train positive: 2988 (7.78%)
  Val positive: 429 (7.84%)

Total validation samples: 5470
Created distributed validation loader, each rank processes ~1284 samples


2025-06-10 13:43:24,276 - INFO - using MLP layer as FFN


...

2025-06-10 16:50:06,013 - INFO - Step    9000  Ep 01  loss 0.6661  lr 0.000100  fetch    0.4 ms  compute 1229.2 ms
2025-06-10 16:50:06,017 - INFO - Accuracy:     T0:0.50  T1:0.52  T2:0.51  T3:0.51  T4:0.54  T5:0.75  
2025-06-10 16:50:06,017 - INFO - Sensitivity:  T0:0.24  T1:0.27  T2:0.51  T3:0.56  T4:0.68  T5:1.00  
2025-06-10 16:50:06,017 - INFO - Specificity:  T0:0.77  T1:0.77  T2:0.50  T3:0.45  T4:0.32  T5:0.00  
2025-06-10 16:50:06,017 - INFO - iteration: 9000 | epoch: 0 | loss: 0.6661 | lr: 0.0001 | type: training | acc_task0: 0.5018 | sens_task0: 0.2380 | spec_task0: 0.7691 | acc_task1: 0.5150 | sens_task1: 0.2704 | spec_task1: 0.7747 | acc_task2: 0.5065 | sens_task2: 0.5109 | spec_task2: 0.5017 | acc_task3: 0.5095 | sens_task3: 0.5624 | spec_task3: 0.4478 | acc_task4: 0.5398 | sens_task4: 0.6754 | spec_task4: 0.3214 | acc_task5: 0.7476 | sens_task5: 0.9981 | spec_task5: 0.0032



Step 10000 (Epoch 1):
Loss: 0.7167
Learning rate: 0.000100

Step   10000  Ep 01  loss 0.7167  lr 0.000100  fetch    0.4 ms  compute 1561.6 ms
Accuracy:     T0:0.50  T1:0.52  T2:0.51  T3:0.51  T4:0.54  T5:0.75  
Sensitivity:  T0:0.22  T1:0.26  T2:0.55  T3:0.60  T4:0.69  T5:1.00  
Specificity:  T0:0.79  T1:0.79  T2:0.47  T3:0.40  T4:0.31  T5:0.00  


2025-06-10 17:16:09,118 - INFO - Step   10000  Ep 01  loss 0.7167  lr 0.000100  fetch    0.4 ms  compute 1561.6 ms
2025-06-10 17:16:09,119 - INFO - Accuracy:     T0:0.50  T1:0.52  T2:0.51  T3:0.51  T4:0.54  T5:0.75  
2025-06-10 17:16:09,119 - INFO - Sensitivity:  T0:0.22  T1:0.26  T2:0.55  T3:0.60  T4:0.69  T5:1.00  
2025-06-10 17:16:09,119 - INFO - Specificity:  T0:0.79  T1:0.79  T2:0.47  T3:0.40  T4:0.31  T5:0.00  
2025-06-10 17:16:09,119 - INFO - iteration: 10000 | epoch: 0 | loss: 0.7167 | lr: 0.0001 | type: training | acc_task0: 0.5048 | sens_task0: 0.2172 | spec_task0: 0.7924 | acc_task1: 0.5184 | sens_task1: 0.2591 | spec_task1: 0.7901 | acc_task2: 0.5092 | sens_task2: 0.5469 | spec_task2: 0.4680 | acc_task3: 0.5089 | sens_task3: 0.6026 | spec_task3: 0.4013 | acc_task4: 0.5435 | sens_task4: 0.6885 | spec_task4: 0.3144 | acc_task5: 0.7454 | sens_task5: 0.9983 | spec_task5: 0.0028


Saved model components to:
  Base: base_ep1_it10000.pt
  Aggregator: aggregator_ep1_it10000.pt
Saved checkpoint at epoch 1 to /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/output_ddp_p16_23_1150_ac200_JH_d10_B64/checkpoints/model_ep1_it10000.pt

Step 11000 (Epoch 1):
Loss: 0.5253
Learning rate: 0.000100

Step   11000  Ep 01  loss 0.5253  lr 0.000100  fetch    0.4 ms  compute 1199.6 ms
Accuracy:     T0:0.51  T1:0.52  T2:0.51  T3:0.51  T4:0.55  T5:0.75  
Sensitivity:  T0:0.20  T1:0.24  T2:0.57  T3:0.64  T4:0.69  T5:1.00  
Specificity:  T0:0.81  T1:0.81  T2:0.46  T3:0.37  T4:0.32  T5:0.00  


2025-06-10 17:40:36,985 - INFO - Step   11000  Ep 01  loss 0.5253  lr 0.000100  fetch    0.4 ms  compute 1199.6 ms
2025-06-10 17:40:36,992 - INFO - Accuracy:     T0:0.51  T1:0.52  T2:0.51  T3:0.51  T4:0.55  T5:0.75  
2025-06-10 17:40:36,992 - INFO - Sensitivity:  T0:0.20  T1:0.24  T2:0.57  T3:0.64  T4:0.69  T5:1.00  
2025-06-10 17:40:36,992 - INFO - Specificity:  T0:0.81  T1:0.81  T2:0.46  T3:0.37  T4:0.32  T5:0.00  
2025-06-10 17:40:36,992 - INFO - iteration: 11000 | epoch: 0 | loss: 0.5253 | lr: 0.0001 | type: training | acc_task0: 0.5050 | sens_task0: 0.1992 | spec_task0: 0.8105 | acc_task1: 0.5192 | sens_task1: 0.2442 | spec_task1: 0.8068 | acc_task2: 0.5149 | sens_task2: 0.5667 | spec_task2: 0.4582 | acc_task3: 0.5127 | sens_task3: 0.6351 | spec_task3: 0.3720 | acc_task4: 0.5477 | sens_task4: 0.6936 | spec_task4: 0.3180 | acc_task5: 0.7454 | sens_task5: 0.9984 | spec_task5: 0.0026
